In [1]:
# Cargar el CSV
import pandas as pd
import numpy as np

consumo_granada_original = pd.read_csv('../data/raw/consumo_granada.csv')

# Crear una copia para trabajar
consumo_granada = consumo_granada_original.copy()

In [2]:
# 1. Eliminar duplicados (manteniendo la primera aparición)
consumo_granada = consumo_granada.drop_duplicates(keep='first')

# 2. Resetear el índice (importante porque ahora tendrás saltos como del 100 al 102)
consumo_granada = consumo_granada.reset_index(drop=True)

# 3. Verificación final
print(f'Nuevo tamaño del dataset: {consumo_granada.shape}')
print(f'Duplicados restantes: {consumo_granada.duplicated().sum()}')

Nuevo tamaño del dataset: (1928180, 5)
Duplicados restantes: 0


In [3]:
# Limpiar los nombres de las zonas: eliminar espacios y convertir todo a formato title case con guiones bajos
consumo_granada['zone_name'] = consumo_granada['zone_name'].str.strip()  # Eliminar espacios al inicio y final
consumo_granada['zone_name'] = consumo_granada['zone_name'].str.replace(' ', '_')  # Reemplazar espacios por guiones bajos
consumo_granada['zone_name'] = consumo_granada['zone_name'].str.title()  # Convertir a formato Title Case

In [4]:
# Formatear los decimales de consumption_kwh a 3 decimales, si tiene 2 decimales se le añade un 0 al final.
consumo_granada['consumption_kwh'] = consumo_granada['consumption_kwh'].round(3)

In [5]:
# Eliminar filas con valores nulos en vez de esto interpolamos. 
# consumo_granada = consumo_granada.dropna()
# consumo_granada.reset_index(drop=True, inplace=True)

# Mirar si tiene nulos
#consumo_granada.isnull().sum()

# Mostar que nulos son 
#consumo_granada[consumo_granada.isnull().any(axis=1)]

In [6]:
# Separar timestamp en fecha y hora
consumo_granada['timestamp'] = pd.to_datetime(consumo_granada['timestamp'])
consumo_granada['fecha'] = consumo_granada['timestamp'].dt.date
consumo_granada['hora'] = consumo_granada['timestamp'].dt.time

In [7]:
# Rellenar valores faltantes de consumo con la media histórica
# En lugar de interpolar, usamos la media de la misma hora/mes/zona de todos los años

# 1. Reemplazar 0 por NaN (consideramos 0 como dato faltante)
consumo_granada['consumption_kwh'] = consumo_granada['consumption_kwh'].replace(0.0, np.nan)

# 2. Extraer componentes temporales para agrupar
consumo_granada['hour'] = consumo_granada['timestamp'].dt.hour
consumo_granada['month'] = consumo_granada['timestamp'].dt.month
consumo_granada['day_of_month'] = consumo_granada['timestamp'].dt.day

# 3. Calcular la media histórica por (zona, mes, día, hora)
# Esto calcula la media de todos los años para esa combinación específica
media_historica = consumo_granada.groupby(['zone_name', 'month', 'day_of_month', 'hour'])['consumption_kwh'].transform('mean')

# 4. Rellenar los NaN con la media histórica
consumo_granada['consumption_kwh'] = consumo_granada['consumption_kwh'].fillna(media_historica)

# 5. Si aún quedan NaN (porque no hay datos históricos para esa combinación),
#    usar la media general de esa zona en esa hora
media_zona_hora = consumo_granada.groupby(['zone_name', 'hour'])['consumption_kwh'].transform('mean')
consumo_granada['consumption_kwh'] = consumo_granada['consumption_kwh'].fillna(media_zona_hora)

print(f"✅ Valores faltantes rellenados con media histórica")
print(f"   NaNs restantes en consumption_kwh: {consumo_granada['consumption_kwh'].isna().sum()}")

✅ Valores faltantes rellenados con media histórica
   NaNs restantes en consumption_kwh: 0


In [8]:
# Rellenar valores faltantes de temperatura con la media histórica
# Mismo enfoque: media de la misma hora/mes/zona de todos los años

# 1. Extraer componentes temporales (si no existen ya)
if 'hour' not in consumo_granada.columns:
    consumo_granada['hour'] = consumo_granada['timestamp'].dt.hour
    consumo_granada['month'] = consumo_granada['timestamp'].dt.month
    consumo_granada['day_of_month'] = consumo_granada['timestamp'].dt.day

# 2. Calcular media histórica de temperatura por (zona, mes, día, hora)
media_temp_historica = consumo_granada.groupby(['zone_name', 'month', 'day_of_month', 'hour'])['temperature'].transform('mean')

# 3. Rellenar NaN con media histórica
consumo_granada['temperature'] = consumo_granada['temperature'].fillna(media_temp_historica)

# 4. Fallback: si aún quedan NaN, usar media de esa zona en esa hora
media_temp_zona_hora = consumo_granada.groupby(['zone_name', 'hour'])['temperature'].transform('mean')
consumo_granada['temperature'] = consumo_granada['temperature'].fillna(media_temp_zona_hora)

print(f"✅ Valores faltantes de temperatura rellenados")
print(f"   NaNs restantes en temperature: {consumo_granada['temperature'].isna().sum()}")

✅ Valores faltantes de temperatura rellenados
   NaNs restantes en temperature: 0


In [9]:
# Guardar el DataFrame limpio en un nuevo archivo CSV
consumo_granada.to_csv('../data/processed/consumo_granada_cleaned.csv', index=False)